In [1]:
import pandas as pd
import re
import numpy as np

In [2]:
df = pd.read_csv("day10.txt", header=None)
df.columns = ["coding"]
df

,coding
0,F-L----F7-7.F-7JJ.F--L-.F|F77.L|7.FLF-7.JJ--7F...
1,J|F7.LL|.F7.FL|J-77--JJ7L|-|-F-7L7-|LJ|F|JL|L-...
2,FF7J.7L|-7|FF7|L7|L|.LF7-JFLJJ.J7JJFJ-7.L-LLJJ...
3,FJ.LF-7-FJLJ-|7-7J777..|-FJ7.FJ.|.F|LFL-L7L|.....
4,|FF7LFL7LJJL|.|.||L77--|7F---L.F|77JFL7.F|7FL7...
...,...
135,|-|||JF-7-7||LJ7J-|7|..F7LFJ7.F7F|.F7.---7||J....
136,|.LFJ-|JLF7-7-||L|||J.FFJ7.|F-7J|7.LL-7|FJF-7F...
137,F-|.F7|.FJ7FL-|-7F||L-J|F7F-L7J-7--F7FFLJLFFL7...
138,.FJF|JFFJLL77||L-7LJ77F-||L-|J.||J.L7-7-L--77F...


In [3]:
x_dim = df.coding.str.len()[0]
y_dim = len(df)
x_dim, y_dim

(140, 140)

In [4]:
symbols = np.zeros((y_dim, x_dim), dtype="str")

In [5]:
starting_node = None

In [6]:
numbers = []
for y, row in enumerate(df.coding):
    for x, val in enumerate(row):
        symbols[y, x] = val
        if val == "S":
            starting_node = y, x
symbols

array([['F', '-', 'L', ..., '-', '7', '-'],
       ['J', '|', 'F', ..., '7', '.', '|'],
       ['F', 'F', '7', ..., '7', 'F', '-'],
       ...,
       ['F', '-', '|', ..., '|', 'J', '7'],
       ['.', 'F', 'J', ..., '7', '|', '|'],
       ['F', '-', 'J', ..., '.', '-', 'J']], dtype='<U1')

In [7]:
starting_node

(25, 108)

In [8]:
visited_nodes = {starting_node: 0}

In [9]:
from collections import deque
queue = deque()
queue.append(starting_node)

In [10]:
class SymbolDefinition:
    def __init__(self, symbol, new_locations_generator) -> None:
        self.symbol = symbol
        self.new_locations_generator = new_locations_generator

    def get_new_locations(self, location):
        y, x = location
        return self.new_locations_generator(y, x)

In [11]:
definitions = [
SymbolDefinition("-", lambda y, x: [(y, x-1), (y, x+1)]),
SymbolDefinition("|", lambda y, x: [(y+1, x), (y-1, x)]),
SymbolDefinition("L", lambda y, x: [(y, x+1), (y-1, x)]),
SymbolDefinition("J", lambda y, x: [(y, x-1), (y-1, x)]),
SymbolDefinition("7", lambda y, x: [(y, x-1), (y+1, x)]),
SymbolDefinition("F", lambda y, x: [(y, x+1), (y+1, x)])
]

In [12]:
symbol_def_map = {symboldef.symbol: symboldef for symboldef in definitions}

In [13]:
def process_locations(new_locations, prev_x, prev_y):
    current_steps = visited_nodes[(prev_y, prev_x)]
    for new_loc in new_locations:
        if new_loc not in visited_nodes:
            visited_nodes[new_loc] = current_steps + 1
            queue.append(new_loc)

while len(queue):
    current_search = queue.popleft()
    y, x = current_search
    #print(f"processing {current_search}")
    symbol = symbols[y, x]
    if symbol == "S":
        if symbols[y, x+1] == "-":
            visited_nodes[(y, x+1)] = 1
            queue.append((y, x+1))
        if symbols[y, x-1] == "-":
            visited_nodes[(y, x-1)] = 1
            queue.append((y, x-1))
        if symbols[y+1, x] == "|":
            visited_nodes[(y+1, x)] = 1
            queue.append((y+1, x))
        if symbols[y-1, x] == "|":
            visited_nodes[(y-1, x)] = 1
            queue.append((y-1, x))
        if symbols[y-1, x] == "J":
            visited_nodes[(y-1, x)] = 1
            queue.append((y-1, x))
        if symbols[y, x+1] == "J":
            visited_nodes[(y, x+1)] = 1
            queue.append((y, x+1))
        if symbols[y+1, x] == "L":
            visited_nodes[(y+1, x)] = 1
            queue.append((y+1, x))
        if symbols[y, x+1] == "7":
            visited_nodes[(y, x+1)] = 1
            queue.append((y, x+1))
        if symbols[y, x-1] == "F":
            visited_nodes[(y, x-1)] = 1
            queue.append((y, x-1))
        print(queue)
    else:
        symbol_def = symbol_def_map[symbol]
        new_locations = symbol_def.get_new_locations(current_search)
        process_locations(new_locations, x, y)   



deque([(24, 108), (26, 108)])


In [14]:
#visited_nodes

In [15]:
#for visited_loc in visited_nodes.keys():
#symbols = np.zeros((y_dim, x_dim), dtype="str")

In [16]:
np.set_printoptions(linewidth=100000)

In [17]:
symbols

array([['F', '-', 'L', ..., '-', '7', '-'],
       ['J', '|', 'F', ..., '7', '.', '|'],
       ['F', 'F', '7', ..., '7', 'F', '-'],
       ...,
       ['F', '-', '|', ..., '|', 'J', '7'],
       ['.', 'F', 'J', ..., '7', '|', '|'],
       ['F', '-', 'J', ..., '.', '-', 'J']], dtype='<U1')

In [18]:
symbols_extended = np.full((2*y_dim-1, 2*x_dim-1), 'X', dtype="str")
for iy, ix in np.ndindex(symbols.shape):
    if iy == 2*y_dim:
        continue
    if ix == 2*x_dim:
        continue
    symbols_extended[2*iy, 2*ix] = symbols[iy, ix]
symbols_extended

array([['F', 'X', '-', ..., '7', 'X', '-'],
       ['X', 'X', 'X', ..., 'X', 'X', 'X'],
       ['J', 'X', '|', ..., '.', 'X', '|'],
       ...,
       ['.', 'X', 'F', ..., '|', 'X', '|'],
       ['X', 'X', 'X', ..., 'X', 'X', 'X'],
       ['F', 'X', '-', ..., '-', 'X', 'J']], dtype='<U1')

In [19]:
# main loop
for visited_loc in visited_nodes.keys():
    y, x = visited_loc
    symbol = symbols[visited_loc]
    symbols_extended[2*y, 2*x] = 'B'
    if symbol == 'S':
        continue
    symbol_def = symbol_def_map[symbol]
    new_locations = symbol_def.get_new_locations(visited_loc)
    for new_location in new_locations:
        y2, x2 = new_location
        symbols_extended[2*y2, 2*x2] = 'B'
        if y==y2:
            if x>x2:
                 symbols_extended[2*y2, 2*x2+1] = 'B'
            else:
                 symbols_extended[2*y2, 2*x+1] = 'B'
        if x==x2:
            if y>y2:
                 symbols_extended[2*y2+1, 2*x2] = 'B'
            else:
                 symbols_extended[2*y+1, 2*x2] = 'B'
symbols_extended

array([['F', 'X', '-', ..., '7', 'X', '-'],
       ['X', 'X', 'X', ..., 'X', 'X', 'X'],
       ['J', 'X', '|', ..., '.', 'X', '|'],
       ...,
       ['.', 'X', 'F', ..., '|', 'X', '|'],
       ['X', 'X', 'X', ..., 'X', 'X', 'X'],
       ['F', 'X', '-', ..., '-', 'X', 'J']], dtype='<U1')

In [20]:
symbols_extended.shape

(279, 279)

In [21]:
def safe_access(matrix, y, x):
    sh = symbols_extended.shape
    y_max = symbols_extended.shape[0]
    x_max = symbols_extended.shape[1]
    if x < 0 or y < 0:
        return
    if x >= x_max or y >= y_max:
        return
    return matrix[y, x]

In [22]:
queue = deque()
reached = set()
for iy, ix in np.ndindex(symbols_extended.shape):
    symb = symbols_extended[iy, ix]
    if symb != 'B':
        if iy == 0 or ix == 0 or iy == symbols_extended.shape[0] -1 or ix == symbols_extended.shape[1] -1:
            queue.append((iy, ix))
            reached.add((iy, ix))
            symbols_extended[iy, ix] = "O"
while len(queue):
    current_search = queue.popleft()
    y, x = current_search    
    up = safe_access(symbols_extended, y+1, x)
    if up and up != 'B':
       new_loc = (y+1, x)
       if new_loc not in reached:
        queue.append(new_loc)
        reached.add(new_loc)
        symbols_extended[y+1, x] = "O" 
    down = safe_access(symbols_extended, y-1, x)
    if down and down != 'B':
       new_loc = (y-1, x)
       if new_loc not in reached:
        queue.append(new_loc)
        reached.add(new_loc)
        symbols_extended[y-1, x] = "O" 
    left = safe_access(symbols_extended, y, x-1)
    if left and left != 'B':
       new_loc = (y, x-1)
       if new_loc not in reached:
        queue.append(new_loc)
        reached.add(new_loc)
        symbols_extended[y, x-1] = "O" 
    right = safe_access(symbols_extended, y, x+1)
    if right and right != 'B':
       new_loc = (y, x+1)
       if new_loc not in reached:
        queue.append(new_loc)
        reached.add(new_loc)
        symbols_extended[y, x+1] = "O" 


In [23]:
symbols_extended

array([['O', 'O', 'O', ..., 'O', 'O', 'O'],
       ['O', 'O', 'O', ..., 'O', 'O', 'O'],
       ['O', 'O', 'O', ..., 'O', 'O', 'O'],
       ...,
       ['O', 'O', 'O', ..., 'O', 'O', 'O'],
       ['O', 'O', 'O', ..., 'O', 'O', 'O'],
       ['O', 'O', 'O', ..., 'O', 'O', 'O']], dtype='<U1')

In [24]:
np.unique(symbols_extended, return_counts=True)

(array(['-', '.', '7', 'B', 'F', 'J', 'L', 'O', 'X', '|'], dtype='<U1'),
 array([   47,    35,    46, 26804,    39,    54,    46, 36426, 14308,    36]))

In [25]:
res = 0
for iy, ix in np.ndindex(symbols_extended.shape): 
    if iy % 2 == 0 and ix % 2 == 0:
        symb = symbols_extended[iy, ix]
        if symb != 'B' and symb != 'O' and symb != 'X':
            res +=1

In [26]:
res

303